# 684 - Redundant Connection

Goal: given an undirected graph that started as a tree with **n** nodes and then had one extra edge added, return that extra edge (the one that creates a cycle). If there are multiple, return the one that appears last in the input.

## Example 1

```
edges = [[1,2],[1,3],[2,3]]
```

Step-by-step graph building:

- Add edge (1,2) → connects 1 and 2

- Add edge (1,3) → connects 1 and 3

- Add edge (2,3) → ⚠️ but 2 and 3 are already connected through 1 (path 2 → 1 → 3) → this creates a cycle.

```
   1
  / \
 2---3
```
Output:
```
[2,3]
```

## Example 2

```
edges = [[1,2],[2,3],[3,4],[1,4],[1,5]]
```
Step-by-step graph building:

- (1,2) → ok

- (2,3) → ok

- (3,4) → ok
(Now: 1–2–3–4 is a chain)

- (1,4) → ⚠️ forms a cycle: 1 → 2 → 3 → 4 → 1

- (1,5) → would be fine, but problem says return the first edge that forms the cycle → that is (1,4).

```
    2
   / \
  1   3
   \ /
    4
    |
    5
```
Output:
```
[1,4]
```

# Base Version

In [2]:
from typing import List

class Solution:
    def findRedundantConnection(self, edges: List[List[int]]) -> List[int]:
        n = len(edges)                          # number of edges (nodes are labeled 1..n)
        parent = [i for i in range(n + 1)]      # each node is initially its own parent
        rank = [0] * (n + 1)                    # rank helps keep tree shallow (union by rank)

        # find function: returns root of a node with path compression
        def find(x: int) -> int:
            if parent[x] != x:
                parent[x] = find(parent[x])     # compress path
            return parent[x]

        # union function: merges two sets, returns False if they were already connected
        def union(a: int, b: int) -> bool:
            ra, rb = find(a), find(b)
            if ra == rb:                        # if roots are same, cycle detected
                return False
            if rank[ra] < rank[rb]:             # attach smaller rank under larger
                parent[ra] = rb
            elif rank[ra] > rank[rb]:
                parent[rb] = ra
            else:
                parent[rb] = ra
                rank[ra] += 1
            return True

        # process each edge; the first that forms a cycle is redundant
        for u, v in edges:
            if not union(u, v):
                return [u, v]

        return []  # should never reach here for valid inputs

sol = Solution()
print(sol.findRedundantConnection([[1,2],[1,3],[2,3]]))  # Output: [2,3]

[2, 3]


# Verbose Version

In [3]:
from typing import List

class Solution:
    def findRedundantConnection(self, edges: List[List[int]]) -> List[int]:
        n = len(edges)  # nodes are labeled 1..n
        parent = [i for i in range(n + 1)]
        rank = [0] * (n + 1)

        print("Input edges:", edges)
        print("Initial parent:", parent)
        print("Initial rank:  ", rank, "\n")

        def find(x: int) -> int:
            # Standard find with path compression, printed verbosely
            if parent[x] != x:
                print(f"find({x}): parent[{x}]={parent[x]} -> compressing path...")
                parent[x] = find(parent[x])
                print(f"find({x}): compressed, parent[{x}]={parent[x]}")
            return parent[x]

        def union(a: int, b: int) -> bool:
            # Returns True if a merge happened; False if a and b were already connected (cycle)
            ra, rb = find(a), find(b)
            print(f"Union({a}, {b}): root(a)={ra}, root(b)={rb}")
            if ra == rb:
                print(f"  -> Cycle detected when adding edge ({a}, {b})!")
                return False

            # Union by rank (attach smaller tree to larger)
            if rank[ra] < rank[rb]:
                parent[ra] = rb
                print(f"  -> Attach root {ra} under root {rb}")
            elif rank[ra] > rank[rb]:
                parent[rb] = ra
                print(f"  -> Attach root {rb} under root {ra}")
            else:
                parent[rb] = ra
                rank[ra] += 1
                print(f"  -> Attach root {rb} under root {ra} and increment rank[{ra}] -> {rank[ra]}")

            print("  Parent:", parent)
            print("  Rank:  ", rank, "\n")
            return True

        # Process edges in order; the first one that can't be merged is redundant
        for idx, (u, v) in enumerate(edges, 1):
            print(f"Processing edge #{idx}: ({u}, {v})")
            if not union(u, v):
                print("\nAnswer (redundant edge):", [u, v])
                return [u, v]

        print("\nNo redundant edge found (should not happen for valid inputs).")
        return []

sol = Solution()
print(sol.findRedundantConnection([[1,2],[1,3],[2,3]]))  # Expected: [2,3]

Input edges: [[1, 2], [1, 3], [2, 3]]
Initial parent: [0, 1, 2, 3]
Initial rank:   [0, 0, 0, 0] 

Processing edge #1: (1, 2)
Union(1, 2): root(a)=1, root(b)=2
  -> Attach root 2 under root 1 and increment rank[1] -> 1
  Parent: [0, 1, 1, 3]
  Rank:   [0, 1, 0, 0] 

Processing edge #2: (1, 3)
Union(1, 3): root(a)=1, root(b)=3
  -> Attach root 3 under root 1
  Parent: [0, 1, 1, 1]
  Rank:   [0, 1, 0, 0] 

Processing edge #3: (2, 3)
find(2): parent[2]=1 -> compressing path...
find(2): compressed, parent[2]=1
find(3): parent[3]=1 -> compressing path...
find(3): compressed, parent[3]=1
Union(2, 3): root(a)=1, root(b)=1
  -> Cycle detected when adding edge (2, 3)!

Answer (redundant edge): [2, 3]
[2, 3]
